<a href="https://colab.research.google.com/github/lmoss/onesharp/blob/main/two_by_two.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>